In [180]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
import astropy.units as u
from icecream import ic, install

import astropaul.targetlistcreator as tlc
import astropaul.html as html
import astropaul.priority as pr
import pandas as pd

install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [181]:
from astroquery.mast import Tesscut
import astropy.units as u
from astropy.coordinates import SkyCoord

coord = SkyCoord(ra=[12.3, 23.4], dec=[45.6, 56.7], unit=u.deg)
for c in coord:
    print(", ".join(Tesscut.get_sectors(coordinates=c)["sector"].astype(str)))


17, 57, 84, 85
18, 58, 85


In [182]:
conn = connect("../../django/TargetDB/db.sqlite3")

name = "TESS Cycle 8 Targets"
creator = tlc.TargetListCreator(name=name, connection=conn)
creator.steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="TESS "),
    tlc.add_speckle,
    # tlc.add_lists,
    # tlc.add_tess_sectors,
    # partial(tlc.filter_targets, criteria=lambda df: (df["Vmag"] < 13)),
    # partial(tlc.filter_targets, inverse=True, criteria=lambda df: (df["List HQND"])),
]
tl = creator.calculate(verbose=False)


# html.render_observing_pages(tl, None, {}, "html")
print(tl.summarize())
coords = tl.target_list.copy()[["Target Name", "ra", "dec"]]
coords["Target Name"] = [name[4:] for name in coords["Target Name"]]
coords.to_csv("coords.csv", index=False, header=False, sep=" ")

TESS Cycle 8 Targets
Criteria
    lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
198 targets:
     195 QuadEB
       3 SextEB
Column Count (primary, secondary):
    Target: (1, 2)
    Coordinates: (6, 3)
    Count: (1, 0)
Associated tables:
     154 rows,  6 columns: Speckle Observations



In [183]:
# hit up the MAST server to find sectors for each target in csv file
# !py .\tess_stars2px.py -f .\coords.csv > sectors.psv

In [184]:
sectors = pd.read_csv("sectors.psv", sep="|")
sectors.columns = [column.strip() for column in sectors.columns]
sectors.columns = ["Target Name"] + list(sectors.columns[1:])
sectors["Target Name"] = [f"TIC {id}" for id in sectors["Target Name"]]
# [", ".join(s["Sector"]) for s in sectors.groupby("Target Name")]
sector_list = pd.DataFrame(sectors.groupby("Target Name")["Sector"].apply(lambda x: ", ".join(map(str, x))))
sector_list.columns = ["Sectors"]
sector_list.reset_index(inplace=True)
tl.target_list = tl.target_list.merge(sector_list, on="Target Name", how="left")
tl.target_list

,Target Name,Target Source,Target Type,ra,dec,pmra,pmdec,parallax,Vmag,Teff,RA,Dec,Num Speckle,Sectors
0,TIC 270360534,Kostov 2023 arXiv:2309.14200,QuadEB,263.430502,43.860752,-11.961600,-23.987400,1.744340,12.041,5048.88,17:33:43.32,+43:51:38.71,1,"25, 26, 40, 52, 53, 78, 79, 80"
1,TIC 219469945,Kostov 2022 arXiv:2202.05790,QuadEB,241.047908,43.030301,-0.866116,-13.091300,1.083280,12.507,6322.79,16:04:11.50,+43:01:49.08,3,"24, 25, 50, 51, 77, 78"
2,TIC 20212631,Kostov 2023 arXiv:2309.14200,QuadEB,227.038060,39.970240,-82.825300,60.712800,5.367930,10.580,5665.40,15:08:09.13,+39:58:12.86,1,"23, 24, 50, 51, 77, 78"
3,TIC 150055835,Kostov 2023 arXiv:2309.14200,QuadEB,69.925310,29.134584,1.092150,-1.994130,0.347484,15.116,6445.00,04:39:42.07,+29:08:04.50,0,"19, 43, 44, 70, 71"
4,TIC 161043618,Kostov 2022 arXiv:2202.05790,QuadEB,223.425163,52.715848,2.178010,-3.594290,2.027610,12.497,5860.00,14:53:42.04,+52:42:57.05,5,"16, 22, 23, 49, 50, 76, 77"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,TIC 414969157,Kostov 2022 arXiv:2202.05790,QuadEB,141.176162,22.200757,-0.138778,-0.350210,1.274850,14.508,5358.36,09:24:42.28,+22:12:02.73,1,"21, 44, 45, 46, 48, 72"
194,TIC 27543409,Kostov 2022 arXiv:2202.05790,QuadEB,122.702004,13.567217,-3.266660,0.942613,0.518392,13.663,6421.00,08:10:48.48,+13:34:01.98,1,"7, 34, 44, 45, 46, 71, 72"
195,TIC 139914081,Kostov 2023 arXiv:2309.14200,QuadEB,326.971651,-49.037832,-4.706880,-3.407270,1.214990,14.534,5229.00,21:47:53.20,-49:02:16.20,0,"1, 28, 68, 95, 105"
196,TIC 382182610,Kostov 2023 arXiv:2309.14200,QuadEB,304.472082,-36.204657,4.584960,-5.932050,1.113940,12.609,6256.14,20:17:53.30,-36:12:16.77,0,"27, 67, 94, 105"


In [ ]:
# anything from sector 97 on is cycle 8 according to https://heasarc.gsfc.nasa.gov/docs/tess/sector.html
tl.target_list["TESS Cycle 8"] = tl.target_list["Sectors"].apply(
    lambda x: any(sector > 97 for sector in list(map(int, x.split(", "))))
)
tl.target_list = tl.target_list[tl.target_list["TESS Cycle 8"]]
print(len(tl.target_list))
html.render_observing_pages(tl, None, {}, "../../Observing Files/TESS Cycle 8 Proposal")


54


In [196]:
proposal_targets = tl.target_list[["Target Name", "ra", "dec", "pmra", "pmdec", "Vmag"]].copy()
proposal_targets.columns = ["TIC Number", "RA", "Dec", "Proper Motion-R.A.", "Proper Motion-Dec.", "TESS Magnitude"]
proposal_targets["TIC Number"] = [target_name[4:] for target_name in proposal_targets["TIC Number"]]
proposal_targets["Target Name"] = [f"TIC {number}" for number in proposal_targets["TIC Number"]]
proposal_targets["Extended?"] = "No"
proposal_targets["Special Handling?"] = "No"
proposal_targets["20-Second Cadence Postage Stamp?"] = "No"
proposal_targets["20-Second Cadence Postage Stamp?"] = "No"
proposal_targets["Swift Exposure Time"] = ""
proposal_targets["NICER Exposure Time"] = ""
proposal_targets["Remarks"] = ""
proposal_targets.to_csv("Proposal Targets.csv", index=False)
print(len(proposal_targets))
proposal_targets

54


,TIC Number,RA,Dec,Proper Motion-R.A.,Proper Motion-Dec.,TESS Magnitude,Target Name,Extended?,Special Handling?,20-Second Cadence Postage Stamp?,Swift Exposure Time,NICER Exposure Time,Remarks
49,60584909,96.619265,-22.184408,-1.893020,1.386280,15.6930,TIC 60584909,No,No,No,,,
50,13021681,75.534801,-24.061641,-2.862850,4.032150,15.3170,TIC 13021681,No,No,No,,,
51,146435300,75.327853,-23.772725,-7.300000,27.300000,11.9610,TIC 146435300,No,No,No,,,
52,93373156,86.750396,-20.903570,1.243630,-3.817710,14.6870,TIC 93373156,No,No,No,,,
53,24700485,81.204640,-7.592358,4.823490,-5.121530,12.7830,TIC 24700485,No,No,No,,,
67,43472931,74.681967,-10.755805,1.029490,-7.523500,14.8080,TIC 43472931,No,No,No,,,
68,200094011,86.782967,0.298943,-0.100000,0.400000,9.7000,TIC 200094011,No,No,No,,,
69,244279814,74.762502,-2.111039,4.484040,0.712685,13.9350,TIC 244279814,No,No,No,,,
71,454140642,64.773473,0.900042,2.827360,-10.846600,10.4090,TIC 454140642,No,No,No,,,
72,459959916,71.489665,4.829619,2.917450,5.876580,13.3230,TIC 459959916,No,No,No,,,
